# 교차검증과 그리드 서치
- 검증세트와 교차검증
- 그리드 서치와 랜덤서치

# Module loading

In [1]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install mglearn
import mglearn

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 7.6 MB/s eta 0:00:00


# Dataset loading

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

# Preprocessing

In [4]:
from sklearn.model_selection import train_test_split

# train / test 분리
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# train -> train / valid 분리
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape)
print(val_input.shape)

(4157, 3)
(1040, 3)


# Modeling
- 검증세트로 모델 평가

In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차검증: K-폴드 교차검증
- 훈련 세트를 k개의 부분으로 나누어서 교차검증을 수행하는 것
- cross_validate() 교차검증 함수 사용, 기본값 cv 매개변수 = 5
  - 일반적으로 5나 10을 사용(데이터가 많으면 10)

In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)  # 검증 데이터(valid)로 쪼개지지 않은 훈련데이터(train) 사용
## fit_time, score_time, test_score
print(scores)
## 딕셔너리 형태로 반환

{'fit_time': array([0.04194999, 0.09226274, 0.03857803, 0.02521467, 0.0106914 ]), 'score_time': array([0.00210285, 0.00382519, 0.00162864, 0.00181532, 0.0018146 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


- test_score의 평균을 구해 최종 점수를 알아냄

In [8]:
np.mean(scores['test_score'])  # 최종 검증결과

0.855300214703487

- train_test_split 함수는 훈련세트를 분리할 떄 섞어서 분리하지만, cross_validate 함수는 훈련세트를 섞어서 폴드를 나누지 않는다.
- 회귀모델 KFold 분할기, 분류모델 StratifiedKFold 분할기 사용

In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 훈련셋트를 섞고 10-폴드 교차검증을 수행

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
# fit_time, score_time, test_score
print(np.mean(scores['test_score']))

0.8574181117533719


#  하이퍼파라미터 튜닝

- 하이퍼파라미터: 사용자가 지정해야만 하는 파라미터
- 파라미터: 모델이 학습을 통해 사용하는 기본 파라미터
- 모델 파라미터: 모델이 학습한 후 취득하게 되는 파라미터, ex) 속성명_
- 하이퍼파라미터 튜닝하는 법
  - 먼저 라이브러리가 제공하는 기본값으로 모델 훈련
  - 검증세트의 점수나 교차검증을 통해 매개변수를 조금씩 바꿔본다
  - 모델마다 적게는 1-2개에서 많게는 5-6개의 매개변수가 제공된다.

# 그리드서치
- 모델의 성능을 가장 높게 하는 최적의 하이퍼파라미터를 찾는 방법
- 하이퍼파라미터 후보들을 하나씩 입력해 모델의 성능을 가장 좋게 만드는 값을 찾는다.
- sklearn.model_selection.GridSearchCV: 시도해볼 하이퍼파라미터들을 지정하면 모든 조합에 대해 교차검증 후 제일 좋은 성능을 내는 하이퍼파라미터 조합을 찾아준다.
- 함수
  - fit(X, y): 학습
  - predict(X): 분류-추론한 class, 회귀-추론한 값
  - predict_proba(X): 분류문제에서 class 별 확률을 반환

  - cv_result_: 파라미터 조합별 평가 결과를 Dictionary로 반환
  - best_params_: 가장 좋은 성능을 낸 파라미터 조합을 반환
  - best_estimator_: 가장 좋은 성능을 낸 모델 반환
  - best_score_: 가장 좋은 점수 반환
- 시도할 하이퍼파라미터와 값들이 너무 많아지면 많은 시간이 소모된다.
  - ex) 1번 파라미터: 10개, 2번 파라미터: 5개, 3번 파라미터: 5개
  - 10 * 5 * 5 = 250개 조합

In [11]:
from sklearn.model_selection import GridSearchCV

# min_impurity_decrease: 노드를 나누기 위한 최소 샘플 수
# max_depth: 트리의 깊이 (default = None)
# min_samples_split: 노드를 분리하기 위한 최소 샘플 수
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004]}

In [14]:
# n_jobs: 병렬 실행에 사용할 CPU 코어 수를 지정하는 것(기본값:1)
# n_jobs = -1 시스템에 있는 모든 코어를 사용한다.
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004]})

In [15]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.9615162593804117
0.8653846153846154


In [16]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [17]:
gs.cv_results_

{'mean_fit_time': array([0.01137681, 0.00805702, 0.01016016, 0.00969844]),
 'std_fit_time': array([0.00340014, 0.00070956, 0.00370897, 0.00393947]),
 'mean_score_time': array([0.0016325 , 0.00142212, 0.00481052, 0.00149012]),
 'std_score_time': array([6.72210857e-05, 6.76933118e-05, 3.37638925e-03, 1.27865287e-04]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154]),
 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161694]),
 'split3_test_score': array([0.86717998, 0.85466795, 0.85081809, 0.84889317]),
 'split4_test_score': 

In [18]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891])

In [20]:
params = {'max_depth':range(5, 20, 1),
          'min_samples_split': range(2, 100, 10),
          'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [21]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
np.max(gs.cv_results_['mean_test_score'])

0.8683865773302731

In [23]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
best_index

841

In [24]:
gs.cv_results_['params'][best_index]

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

# 랜덤 서치(Random Search)

 - 매개변수의 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달
 - from scipy.stats import uniform, randint
 - uniform: 균등분포 샘플링
 - randint(), rvs()
 - 테스트 세트의 점수는 검증세트에 대한 점수보다 조금 낮은 것이 일반적이다.

In [25]:
from scipy.stats import uniform, randint

In [26]:
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 4, 0, 2, 4, 1, 1, 3, 8, 5])

In [27]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 87,  95, 107,  99,  95, 102,  99, 107, 115,  94]))

In [29]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.42693678, 0.36593864, 0.03548931, 0.83009972, 0.90650735,
       0.79108665, 0.92636601, 0.55453969, 0.75046984, 0.10378487])

In [30]:
params = {'max_depth':randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_impurity_decrease': uniform(0.0001, 0.001),
          'min_samples_leaf': randint(1, 25)}

In [31]:
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                        params, n_iter=100, n_jobs=-1, random_state=42)

rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7897e426ac50>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7897e426b070>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7897e426a470>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7897e472e5f0>},
                   random_state=42)

In [32]:
rs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [33]:
# 교차검증 최고 점수
np.max(rs.cv_results_['mean_test_score'])

0.8695428296438884

In [35]:
# 최고의 학습 모델
dt = rs.best_estimator_
print(dt.score(test_input, test_target))
print(dt.score(train_input, train_target))
## 과대적합 해결
## but 과소적합 발생

0.86
0.8928227823744468
